In [84]:
import numpy as np
import pandas as pd
from os import getcwd
from os.path import dirname 

from neuromaps.images import load_data, load_gifti, annot_to_gifti, relabel_gifti, construct_shape_gii
from neuromaps.datasets import fetch_annotation
from neuromaps.resampling import resample_images
from neuromaps.nulls import alexander_bloch, burt2020
from neuromaps.parcellate import Parcellater
from scipy.stats import pearsonr
from neuromaps import transforms 
from neuromaps.stats import compare_images
from neuromaps.nulls import hungarian


import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [86]:

# define path
path = dirname(getcwd()) + '/'

# load in different parcellation files
dk_fsaverage_10k = (path + 'parcellations/atlas-desikankilliany_space-fsaverage_den-10k_hemi-L.label.gii.gz',
                     path + 'parcellations/atlas-desikankilliany_space-fsaverage_den-10k_hemi-R.label.gii.gz')
dk_fsaverage_164k = (path + 'parcellations/atlas-desikankilliany_space-fsaverage_den-164k_hemi-L.aparc-1.annot',
                     path + 'parcellations/atlas-desikankilliany_space-fsaverage_den-164k_hemi-R.aparc-1.annot')
dk_mni = path + 'parcellations/atlas-desikankilliany_space-MNI_res-1mm.nii.gz'

# make sure label IDs are consecutive across hemispheres
dk_fsaverage_10k = relabel_gifti(dk_fsaverage_10k)
dk_fsaverage_164k = annot_to_gifti(dk_fsaverage_164k)  # this does relabel_gift and also converts the annot file to gifti

# make the parcellaters for each space
parcellater_fs10k = Parcellater(dk_fsaverage_10k, 'fsaverage')
parcellater_fs164k = Parcellater(dk_fsaverage_164k, 'fsaverage')
parcellater_mni = Parcellater(dk_mni, 'MNI152')

In [88]:
# download enigma
enigmamap = pd.read_csv(path+'data/ENIGMA_S32_partial_correlation_between_cortical_thickness_and_chlorpromazine_equivalents.csv')
enigmamap.drop([68, 69], inplace=True)  # remove the last two rows
enigma_parc = enigmamap['partial_r'].to_numpy()

# download the regions for MNI152, take indecies of surface rois  
rois = pd.read_csv(path+'parcellations/atlas-desikankilliany.csv')
rois = rois[(rois['structure'] == 'cortex')].index.to_numpy()


In [89]:
# get turku maps  
img_L = load_data(path + 'data/lh.sig.nii')
img_gii_L = construct_shape_gii(img_L)

img_R = load_data(path + 'data/rh.sig.nii')  
img_gii_R = construct_shape_gii(img_R)

#parcellate the turku_map into turku_parc
turku_map = (img_gii_L, img_gii_R)
turku_parc = parcellater_fs164k.fit_transform(turku_map, space='fsaverage', ignore_background_data=True)
np.save(path + 'data/turku_parc.npy', turku_parc)

In [94]:
# get annotations 
annotations = list(fetch_annotation(source=['hcps1200',
                                            'raichle',
                                            'ding2010', 
                                            'finnema2016', 
                                            'dubois2015',
                                            'gallezot2010',
                                            'gallezot2017',
                                            'hillmer2016',
                                            'jaworska2020',
                                            'kaller2017',
                                            'kantonen2020',
                                            'laurikainen2018',
                                            'normandin2015',
                                            'radnakrishnan2018',
                                            'sandiego2015',
                                            'satterthwaite2014',
                                            'savli2012',
                                            'satterthwaite2014',
                                            'smith2017',
                                            'tuominen',
                                            'naganawa2020',
                                            'fazio2016']).keys())

annotations.extend(fetch_annotation(source=['norgaard2021', 'beliveau2017'], space='fsaverage').keys())
annotations.extend(fetch_annotation(source='margulies2016', desc='fcgradient01', return_single=False).keys())


In [91]:
# parcellate annotations

# initialize
parcellated = dict([])

# go over each annotation and parcellate depending on the space 
for (src, desc, space, den) in annotations:

    annot = fetch_annotation(source=src, desc=desc, space=space, den=den)
    
    if space == 'MNI152':
        parcellater = parcellater_mni
    elif space == 'fsaverage' and den == '164k':
        parcellater = parcellater_fs164k
    elif space == 'fsLR' and den == '164k':
        space = 'fsaverage'
        annot = transforms.fslr_to_fsaverage(annot, target_density='164k')
        parcellater = parcellater_fs164k
    elif space == 'fsLR' and den != '164k':
        # unfortunately for fsLR-4k we are upsampling to fsaverage-10k to parcellate but it should be fine
        space = 'fsaverage'
        annot = transforms.fslr_to_fsaverage(annot, target_density='10k')
        parcellater = parcellater_fs10k

    parcellated[desc] = parcellater.fit_transform(annot, space=space, ignore_background_data=True)

    # if subcortex included remove 
    if parcellated[desc].shape == (1,83):
        parcellated[desc] = parcellated[desc][0][rois]
  

/Users/laurituominen/anaconda3/lib/python3.10/site-packages/scipy/ndimage/_measurements.py:803: RuntimeWarning: invalid value encountered in divide
  return sum / numpy.asanyarray(count).astype(numpy.float64)


In [92]:
# get spins 
spins = pd.read_csv(path + 'parcellations/spins_hungarian_aparc+aseg_ctx.csv', header=None)
nspins = spins.values.shape[1]


In [93]:
# Correlations between parcellated annotations & parcellated antipsychotic effects on cortical thickness 
# initialize dictionary to save out later
nulls_enigma = dict([])
corrs_enigma = dict([])

nulls_turku = dict([])
corrs_turku = dict([])

# go over annotations 
for src, desc, space, den in annotations:
    if space == 'MNI152':
        parcellation=dk_mni
        
    elif space == 'fsaverage' and den == '164k':
        parcellation=dk_fsaverage_164k
        
    elif space == 'fsLR' and den == '164k':
        parcellation=dk_fsaverage_164k
        
    elif space == 'fsLR' and den != '164k':
        parcellation=dk_fsaverage_10k
    
    # empirical correlation between annotations, turku and enigma map
    rho_enigma = pearsonr(parcellated[desc], enigma_parc)[0]
    rho_turku = pearsonr(parcellated[desc], turku_parc)[0]
    
    # get 10k rotations 
    rotated = hungarian(data=parcellated[desc], n_perm=10000, spins=spins, parcellation=parcellation) 
    
    # get null Turku
    n = np.zeros((nspins, ))
    for i in range(nspins):
        n[i] = pearsonr(turku_parc, rotated[:,i])[0]    
    
    # get p-value Turku
    pspin = (1 + sum(abs(n) > abs(rho_turku ))) / (nspins + 1)

    # store, multiply by -1 to make more intuitive, because smaller p-value means bigger effect  
    corrs_turku[src+'_'+desc] = ( (-1 * rho_turku, pspin ) )
    nulls_turku[src+'_'+desc] = n
    
    # get null enigma
    n = np.zeros((nspins, ))
    for i in range(nspins):
        n[i] = pearsonr(enigma_parc, rotated[:,i])[0]    
    
    # get p-value enigma
    pspin = (1 + sum(abs(n) > abs(rho_enigma ))) / (nspins + 1)

    # store, multiply by -1 to make more intuitive, because smaller partial r means bigger effect
    corrs_enigma[src+'_'+desc] = ( (-1 * rho_enigma, pspin ) )
    nulls_enigma[src+'_'+desc] = n

# save correlations & nulls 
np.savez(path + 'data/corrs_turku.npz', **corrs_turku)
np.savez(path + 'data/nulls_turku.npz', **nulls_turku)
np.savez(path + 'data/nulls_enigma.npz', **nulls_enigma)
np.savez(path + 'data/corrs_enigma.npz', **corrs_enigma)

In [20]:
subgroups = {'dopamine': ( 'kaller2017_sch23390','jaworska2020_fallypride'),  
             'serotonin': ('beliveau2017_dasb', 'savli2012_way100635','beliveau2017_az10419369', 
                           'beliveau2017_cimbi36', 'beliveau2017_sb207145', 'radnakrishnan2018_gsk215083'), 
            'acetylcholine': ('tuominen_feobv', 'hillmer2016_flubatine', 'naganawa2020_lsn3172176'),
            'function': ('margulies2016_fcgradient01', 'mueller2013_intersubjvar'),
             'neurophysiology': ('hcps1200_megalpha','hcps1200_megbeta', 'hcps1200_megdelta', 
                            'hcps1200_meggamma1', 'hcps1200_meggamma2',
                            'hcps1200_megtheta', 'hcps1200_megtimescale'),
             'structural': ('finnema2016_ucbj', 'hcps1200_thickness', 'hcps1200_myelinmap'),
             'various': ('dukart2018_flumazenil', 'dubois2015_abp688',
                         'laurikainen2018_fmpepd2', 'kantonen2020_carfentanil',
                        'gallezot2017_gsk189254'),
             'metabolic' : ('raichle_cbf', 'raichle_cbv', 'raichle_cmr02', 'raichle_cmruglu')
            }

plot_order = ["dopamine", "serotonin", "acetylcholine", "various", "function", "neurophysiology", 
              "structural", "metabolic"]


In [64]:
sources = [item for value in subgroups.values() for item in value]
sources.remove('mueller2013_intersubjvar')
sources

['kaller2017_sch23390',
 'jaworska2020_fallypride',
 'beliveau2017_dasb',
 'savli2012_way100635',
 'beliveau2017_az10419369',
 'beliveau2017_cimbi36',
 'beliveau2017_sb207145',
 'radnakrishnan2018_gsk215083',
 'tuominen_feobv',
 'hillmer2016_flubatine',
 'naganawa2020_lsn3172176',
 'margulies2016_fcgradient01',
 'hcps1200_megalpha',
 'hcps1200_megbeta',
 'hcps1200_megdelta',
 'hcps1200_meggamma1',
 'hcps1200_meggamma2',
 'hcps1200_megtheta',
 'hcps1200_megtimescale',
 'finnema2016_ucbj',
 'hcps1200_thickness',
 'hcps1200_myelinmap',
 'dukart2018_flumazenil',
 'dubois2015_abp688',
 'laurikainen2018_fmpepd2',
 'kantonen2020_carfentanil',
 'gallezot2017_gsk189254',
 'raichle_cbf',
 'raichle_cbv',
 'raichle_cmr02',
 'raichle_cmruglu']

In [69]:

from statsmodels.stats.multitest import multipletests

df_turku = pd.DataFrame(corrs_turku).transpose()
df_turku.columns = ['rho', 'pspin']
df_turku = df_turku[df_turku.index.isin(sources)]
_, bon_corrected_pvalues, _, _ = multipletests(df_turku['pspin'], alpha=0.05, method='bonferroni')
_, fdr_corrected_pvalues, _, _ = multipletests(df_turku['pspin'], alpha=0.05, method='fdr_bh')

df_turku['bon_corrected_p_value'] = bon_corrected_pvalues
df_turku['fdr_corrected_p_value'] = fdr_corrected_pvalues

df_turku

,rho,pspin,bon_corrected_p_value,fdr_corrected_p_value
dubois2015_abp688,0.118611,0.328367,1.000000,0.407175
dukart2018_flumazenil,0.101912,0.397160,1.000000,0.473537
finnema2016_ucbj,0.300756,0.009499,0.294471,0.022652
gallezot2017_gsk189254,0.074659,0.550645,1.000000,0.609643
hcps1200_megalpha,-0.393699,0.001700,0.052695,0.007971
hcps1200_megbeta,0.213797,0.084292,1.000000,0.130652
hcps1200_megdelta,0.284394,0.014299,0.443256,0.030377
hcps1200_meggamma1,0.423075,0.000600,0.018598,0.007439
hcps1200_meggamma2,0.259833,0.021298,0.660234,0.038837
hcps1200_megtheta,0.376919,0.005000,0.154985,0.015498


In [70]:
index_fdr = df_turku.index[df_turku['fdr_corrected_p_value']< 0.05] 

df_enigma = pd.DataFrame(corrs_enigma).transpose()
df_enigma.columns = ['rho', 'pspin']
df_enigma = df_enigma[df_enigma.index.isin(index_fdr)]
_, bon_corrected_pvalues, _, _ = multipletests(df_enigma['pspin'], alpha=0.05, method='bonferroni')
_, fdr_corrected_pvalues, _, _ = multipletests(df_enigma['pspin'], alpha=0.05, method='fdr_bh')

df_enigma['bon_corrected_p_value'] = bon_corrected_pvalues
df_enigma['fdr_corrected_p_value'] = fdr_corrected_pvalues
df_enigma

,rho,pspin,bon_corrected_p_value,fdr_corrected_p_value
finnema2016_ucbj,0.306844,0.009199,0.165583,0.015053
hcps1200_megalpha,-0.352202,0.005399,0.097190,0.012149
hcps1200_megdelta,0.228598,0.082292,1.000000,0.105804
hcps1200_meggamma1,0.295625,0.018898,0.340166,0.028347
hcps1200_meggamma2,0.198684,0.123488,1.000000,0.138924
hcps1200_megtheta,0.396206,0.001700,0.030597,0.006119
hcps1200_megtimescale,0.178081,0.164984,1.000000,0.164984
hcps1200_myelinmap,-0.270243,0.033997,0.611939,0.047072
hillmer2016_flubatine,0.361201,0.002900,0.052195,0.007456
kantonen2020_carfentanil,0.387682,0.002600,0.046795,0.007456


In [26]:
type(corrs_turku)

dict